In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import lxml
import scipy.stats as st
from bstrap import bootstrap
import numpy as np

In [2]:
url1 = 'https://www.kinopoisk.ru/lists/movies/top250/?page=1'
url2 = 'https://www.kinopoisk.ru/lists/movies/top250/?page=2'
url3 = 'https://www.kinopoisk.ru/lists/movies/top250/?page=3'
url4 = 'https://www.kinopoisk.ru/lists/movies/top250/?page=4'
url5 = 'https://www.kinopoisk.ru/lists/movies/top250/?page=5'

Below we need to start cells with some time interval to avoid blocking from kinopoisk
Maybe it can be possible to add all headers and cookies as we have requests from browser to automate it in one function. However, it was not successful in my realization.

In [3]:
resp1 = requests.get(url1)

In [8]:
resp2 = requests.get(url2)

In [11]:
resp3 = requests.get(url3)

In [12]:
resp4 = requests.get(url4)

In [15]:
resp5 = requests.get(url5)

In [6]:
def data_parser(response):
 soup = BeautifulSoup(response.text, 'lxml')
 films=soup.findAll('div',class_='styles_root__ti07r')
 data =[]

 for film in films:
  rate=film.find('div', class_='styles_root__j_C_S').find('span', class_='styles_position__TDe4E').text
  name=film.find('a',class_='base-movie-main-info_link__YwtP1').find('span', class_='desktop-list-main-info_secondaryText__M_aus').text#.find('span', class_='desktop-list-main-info_secondaryText__M_aus')
  name_rus=film.find('div',class_='styles_content__nT2IG').find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text
  genre=film.find('div',class_='styles_content__nT2IG').findAll('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text
  rating_new=film.find('div',class_='styles_rating__ni2L0 styles_root___s7Tg styles_rootMd__ZvdRj styles_rootPositive__PIwO2').text
  rating_old=film.find('div',class_='styles_user__2wZvH').find('div',class_='styles_rating__LU3_x').find('span',class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg').text
  views=film.find('div',class_='styles_user__2wZvH').find('div',class_='styles_rating__LU3_x').find('span',class_='styles_kinopoiskCount__2_VPQ').text

  data.append([rate,name,name_rus,genre,rating_new,rating_old,views])
  header =['rate', 'name','name_rus','genre','rating_new','rating_old','views']
  df=pd.DataFrame(data,columns=header)

 return df

In [16]:
df1 = data_parser(resp1)
df2 = data_parser(resp2)
df3 = data_parser(resp3)
df4 = data_parser(resp4)
df5 = data_parser(resp5)

In [17]:
df = pd.concat([df1,df2,df3,df4,df5])
df.to_excel('parsed_rating_kinopoisk.xlsx')

Let's check the following hypothesis:

H0 - there is no statistical difference between rating_new and rating_old

Ha - the difference between rating_new and rating_old exists

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 49
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   rate        250 non-null    object 
 1   name        250 non-null    object 
 2   name_rus    250 non-null    object 
 3   genre       250 non-null    object 
 4   rating_new  250 non-null    float64
 5   rating_old  250 non-null    float64
 6   views       250 non-null    object 
dtypes: float64(2), object(5)
memory usage: 15.6+ KB


In [35]:
df.rating_old = df.rating_old.astype('float')
df.rating_new = df.rating_new.astype('float')


In [36]:
df.describe()

,rating_new,rating_old
count,250.000000,250.000000
mean,8.211600,8.317200
std,0.255198,0.207452
min,7.700000,8.100000
25%,8.000000,8.200000
50%,8.200000,8.300000
75%,8.400000,8.400000
max,9.100000,9.200000


In [39]:
df.genre

0                 США • драма  Режиссёр: Фрэнк Дарабонт
1                США • драма  Режиссёр: Стивен Спилберг
2                 США • драма  Режиссёр: Фрэнк Дарабонт
3                 США • драма  Режиссёр: Роберт Земекис
4     Новая Зеландия • фэнтези  Режиссёр: Питер Джексон
                            ...                        
45                 США • драма  Режиссёр: Пенни Маршалл
46               США • драма  Режиссёр: Мартин Скорсезе
47              СССР • вестерн  Режиссёр: Алла Сурикова
48                США • мультфильм  Режиссёр: Ли Анкрич
49            США • боевик  Режиссёр: Квентин Тарантино
Name: genre, Length: 250, dtype: object

In [50]:
df[['country', 'other']] = df.genre.str.split('•', expand=True)
df[['genre_name', 'director']] = df.other.str.split('Режиссёр: ', expand=True)

In [57]:
df.columns

Index(['rate', 'name', 'name_rus', 'genre', 'rating_new', 'rating_old',
       'views', 'country', 'other', 'genre_name', 'director'],
      dtype='object')

In [58]:
df = df[['rate', 'name', 'name_rus', 'genre_name', 'director', 'country', 'rating_old', 'rating_new', 'views']]
df

,rate,name,name_rus,genre_name,director,country,rating_old,rating_new,views
0,1,"The Green Mile, 1999, 189 мин.",Зеленая миля,драма,Фрэнк Дарабонт,США,9.2,9.1,121 625
1,2,"Schindler's List, 1993, 195 мин.",Список Шиндлера,драма,Стивен Спилберг,США,9.1,8.8,76 405
2,3,"The Shawshank Redemption, 1994, 142 мин.",Побег из Шоушенка,драма,Фрэнк Дарабонт,США,9.0,9.1,79 145
3,4,"Forrest Gump, 1994, 142 мин.",Форрест Гамп,драма,Роберт Земекис,США,8.9,8.9,105 284
4,5,"The Lord of the Rings: The Return of the King,...",Властелин колец: Возвращение короля,фэнтези,Питер Джексон,Новая Зеландия,8.9,8.7,50 929
...,...,...,...,...,...,...,...,...,...
45,246,"Awakenings, 1990, 121 мин.",Пробуждение,драма,Пенни Маршалл,США,8.1,8.4,12 727
46,247,"Goodfellas, 1990, 140 мин.",Славные парни,драма,Мартин Скорсезе,США,8.1,8.1,27 746
47,248,"1987, 98 мин.",Человек с бульвара Капуцинов,вестерн,Алла Сурикова,СССР,8.1,8.0,15 964
48,249,"Toy Story 3, 2010, 103 мин.",История игрушек: Большой побег,мультфильм,Ли Анкрич,США,8.1,7.8,27 340


In [60]:
df.describe()

,rating_old,rating_new
count,250.000000,250.000000
mean,8.317200,8.211600
std,0.207452,0.255198
min,8.100000,7.700000
25%,8.200000,8.000000
50%,8.300000,8.200000
75%,8.400000,8.400000
max,9.200000,9.100000


In [61]:
st.mannwhitneyu(df.rating_old, df.rating_new, alternative='two-sided')

MannwhitneyuResult(statistic=40070.5, pvalue=3.267566667722795e-08)

We can see pvalue = 3.27, which is higher than 0.05, that is why we can't reject H0 hypothesis. So there is no statistical difference between rating_new and rating_old

Let's filter films by genre and research "драма" films

In [62]:
drama_films = df[df['genre_name'].str.contains('драма')]
st.mannwhitneyu(drama_films['rating_old'], drama_films['rating_new'], alternative='two-sided')

MannwhitneyuResult(statistic=3422.0, pvalue=0.09426633971206119)

As we can see from the result, it's similar to the previous experiment. pvalue is higher than 0.05, in such case old and new rating do not have statistical difference

We can also try to compare some statistical metrics from the table below, grouped by origin of the films

In [63]:
df.groupby('country').agg({'rating_old': [np.mean, np.max , np.min, np.sum], 
                              'rating_new': [np.mean, np.max , np.min, np.sum]})

rating_old                   rating_new                  
                      mean amax amin     sum       mean amax amin     sum
country                                                                  
Австралия         8.350000  8.5  8.2    16.7   7.950000  8.1  7.8    15.9
Беларусь          8.300000  8.3  8.3     8.3   8.000000  8.0  8.0     8.0
Великобритания    8.295000  8.6  8.1   165.9   8.140000  8.6  7.8   162.8
Германия          8.325000  8.6  8.2    33.3   8.175000  8.6  8.0    32.7
Дания             8.200000  8.2  8.2     8.2   8.000000  8.0  8.0     8.0
Ирландия          8.300000  8.3  8.3     8.3   8.200000  8.2  8.2     8.2
Испания           8.500000  8.5  8.5     8.5   8.600000  8.6  8.6     8.6
Италия            8.400000  8.5  8.3    25.2   8.500000  8.6  8.4    25.5
Корея Южная       8.100000  8.1  8.1     8.1   8.200000  8.2  8.2     8.2
Новая Зеландия    8.550000  8.9  8.2    51.3   8.300000  8.7  7.8    49.8
Россия            8.300000  8.5  8.1    58.1   8.085714  8.3  7.9    56.6
СССР              8.337209  8.7  8.1   358.5   8.318605  8.8  8.0   357.7
США               8.304965  9.2  8.1  1171.0   8.187234  9.1  7.7  1154.4
Франция           8.337500  8.8  8.1    66.7   8.275000  8.8  8.0    66.2
Швеция            8.300000  8.3  8.3     8.3   8.100000  8.1  8.1     8.1
Япония            8.290000  8.7  8.1    82.9   8.220000  8.5  8.1    82.2

Let's use Bootstrap for our hypothesis

In [69]:
old, new, p = bootstrap(np.mean, df['rating_old'], df['rating_new'])

In [70]:
old, new, p

({'avg_metric': 8.317186208, 'metric_ci_lb': 8.296, 'metric_ci_ub': 8.3392},
 {'avg_metric': 8.211599848,
  'metric_ci_lb': 8.1852,
  'metric_ci_ub': 8.238399999999999},
 0.0)

In [67]:
old1, new1, p1 = bootstrap(np.mean, drama_films['rating_old'], drama_films['rating_new'])

In [68]:
old1, new1, p1

({'avg_metric': 8.325948272727274,
  'metric_ci_lb': 8.284415584415585,
  'metric_ci_ub': 8.37012987012987},
 {'avg_metric': 8.268791558441558,
  'metric_ci_lb': 8.220779220779221,
  'metric_ci_ub': 8.31818181818182},
 0.15122)

We can see that again there is no statistical difference between metrics

In [71]:
df.to_excel('clean_rating_kinopoisk.xlsx')